In [26]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [27]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [28]:
tf.keras.backend.clear_session()

In [29]:
subject = 'Mato Grosso - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [30]:
data = pd.read_csv('2003_mo_model_input_MT.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value,Mato Grosso - Consumo de Cimento (t)
0,2003-1,50.917240,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,1.937098,994.641929,1307.693017,26.497759,22.112509,78.548056,2.038260,8.297178,0.745147,6.084564e+07,2.670588e+06,16.831332,5.118946e+07,0.331800,47.470
1,2003-2,50.851586,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,2.057965,994.309470,1283.219898,26.168996,21.331689,76.901685,1.862279,8.291317,0.745308,6.092768e+07,2.672688e+06,16.835967,5.121614e+07,0.333615,45.387
2,2003-3,50.970549,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,1.715492,994.660654,1261.062010,25.844852,20.557682,75.407092,1.802031,8.285457,0.745469,6.100973e+07,2.674788e+06,16.840602,5.124282e+07,0.334864,44.907
3,2003-4,51.049978,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,1.525801,995.244084,1235.995684,25.435732,19.399042,72.894611,1.794850,8.279596,0.745630,6.109177e+07,2.676888e+06,16.845237,5.126950e+07,0.336048,45.467
4,2003-5,51.567529,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,1.166060,995.697467,1233.790572,25.082755,17.830386,68.635209,1.820612,8.273735,0.745791,6.117381e+07,2.678988e+06,16.849872,5.129619e+07,0.336447,56.246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,119.544326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.102263,991.329315,1539.949273,27.574216,15.320440,51.622595,1.565635,NaN,NaN,NaN,NaN,NaN,NaN,0.600607,192.533
236,2022-9,118.223448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.138067,990.426899,1532.187241,27.972317,16.550493,53.885774,1.820854,NaN,NaN,NaN,NaN,NaN,NaN,0.598965,183.895
237,2022-10,117.524152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.925018,990.065020,1555.691347,27.977503,16.972918,55.046118,1.895816,NaN,NaN,NaN,NaN,NaN,NaN,0.596866,180.525
238,2022-11,116.430559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.972980,989.787595,1584.943790,27.954543,16.866632,54.824446,2.022442,NaN,NaN,NaN,NaN,NaN,NaN,0.594246,161.683


In [31]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value
0,0.001009,1.000000,1.000000,0.813935,1.000000,0.000000,0.000000,1.000000,0.336775,0.912870,0.080500,0.517405,1.000000,1.000000,0.943601,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.182887
1,0.000000,0.921240,0.965719,0.698054,0.936603,0.007992,0.011007,0.968938,0.359949,0.887077,0.078989,0.428403,0.903498,0.943223,0.859609,0.996341,0.011805,0.005153,0.008692,0.004513,0.005346,0.195974
2,0.001828,0.873107,0.927412,1.000000,0.906543,0.015984,0.022015,0.948438,0.294285,0.914322,0.077621,0.340651,0.807838,0.891680,0.830854,0.992682,0.023609,0.010307,0.017385,0.009026,0.010692,0.204977
3,0.003049,0.851659,0.885588,0.629650,0.858675,0.023976,0.033022,0.928794,0.257914,0.959586,0.076073,0.229895,0.664640,0.805034,0.827427,0.989023,0.035414,0.015460,0.026077,0.013539,0.016038,0.213516
4,0.011003,0.823277,0.838034,0.795263,0.824839,0.031968,0.044030,0.970421,0.188939,0.994760,0.075937,0.134338,0.470769,0.658143,0.839723,0.985364,0.047219,0.020613,0.034770,0.018052,0.021384,0.216392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.692912,0.000000,0.030808,0.075260,0.381401,0.987600,0.828698,0.005731,0.363180,0.185517,0.034447,0.473616,0.382418,0.412205,0.721547,0.232479,0.768057,0.988798,0.647567,0.992324,0.990966,0.843701
188,0.693045,0.029504,0.027720,0.052572,0.466609,0.990700,0.823090,0.037761,0.457237,0.124386,0.045597,0.583810,0.522215,0.496420,0.710959,0.228761,0.756242,0.987198,0.649169,0.991228,0.989675,0.850718
189,0.701363,0.043125,0.023074,0.013826,0.530936,0.993800,0.817482,0.087044,0.484434,0.111254,0.053649,0.580652,0.685920,0.639336,0.691975,0.225042,0.744427,0.985598,0.650771,0.990131,0.988384,0.864656
190,0.702586,0.059756,0.017547,0.056080,0.611274,0.996900,0.811875,0.118264,0.530068,0.099285,0.056431,0.585672,0.758157,0.707155,0.691303,0.221324,0.732612,0.983998,0.652373,0.989035,0.987094,0.878935


In [32]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      51.972
1      40.729
2      54.446
3      51.788
4      58.702
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Mato Grosso - Consumo de Cimento (t), Length: 240, dtype: float64

In [33]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value
0,0.001009,1.000000,1.000000,0.813935,1.000000,0.000000,0.000000,1.000000,0.336775,0.912870,0.080500,0.517405,1.000000,1.000000,0.943601,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.182887
1,0.000000,0.921240,0.965719,0.698054,0.936603,0.007992,0.011007,0.968938,0.359949,0.887077,0.078989,0.428403,0.903498,0.943223,0.859609,0.996341,0.011805,0.005153,0.008692,0.004513,0.005346,0.195974
2,0.001828,0.873107,0.927412,1.000000,0.906543,0.015984,0.022015,0.948438,0.294285,0.914322,0.077621,0.340651,0.807838,0.891680,0.830854,0.992682,0.023609,0.010307,0.017385,0.009026,0.010692,0.204977
3,0.003049,0.851659,0.885588,0.629650,0.858675,0.023976,0.033022,0.928794,0.257914,0.959586,0.076073,0.229895,0.664640,0.805034,0.827427,0.989023,0.035414,0.015460,0.026077,0.013539,0.016038,0.213516
4,0.011003,0.823277,0.838034,0.795263,0.824839,0.031968,0.044030,0.970421,0.188939,0.994760,0.075937,0.134338,0.470769,0.658143,0.839723,0.985364,0.047219,0.020613,0.034770,0.018052,0.021384,0.216392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.588452,0.379453,0.024030,0.365935,0.039971,0.890665,0.956993,0.182507,0.614556,0.160961,0.040648,0.774460,0.887824,0.741466,0.831428,0.621493,0.994318,0.973043,0.688394,0.986108,0.975683,0.938508
158,0.582813,0.332832,0.021531,0.401765,0.027548,0.893949,0.953662,0.168812,0.616944,0.234357,0.032268,0.483340,0.797305,0.785424,0.791149,0.596535,0.988636,0.974495,0.684427,0.985875,0.976460,0.931775
159,0.576827,0.313954,0.019385,0.379504,0.037038,0.897232,0.950331,0.152959,0.551264,0.283480,0.027411,0.372192,0.601840,0.652419,0.788606,0.571578,0.982954,0.975947,0.680460,0.985641,0.977238,0.924689
160,0.568447,0.294856,0.017671,0.440821,0.070841,0.900516,0.947000,0.137613,0.335005,0.315601,0.022591,0.310244,0.411864,0.510041,0.775390,0.546620,0.977272,0.977399,0.676493,0.985408,0.978015,0.918022


In [34]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       51.972
1       40.729
2       54.446
3       51.788
4       58.702
        ...   
157     65.293
158     90.338
159     83.188
160    107.230
161    105.397
Name: Mato Grosso - Consumo de Cimento (t), Length: 162, dtype: float64

In [35]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [36]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 22)

In [37]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value
126,1.000000,0.612440,0.052718,0.131587,0.201912,0.780557,0.994213,0.372054,0.315314,0.241827,0.051317,0.424482,0.188414,0.254053,0.811122,0.366310,0.798772,0.815216,0.971029,0.870123,0.843018,0.868231
127,0.991476,0.626227,0.056653,0.217266,0.209545,0.784414,0.995177,0.375438,0.345117,0.179021,0.054052,0.535355,0.255572,0.272142,0.834779,0.388500,0.797810,0.822368,0.966201,0.876689,0.849696,0.879874
128,0.976090,0.645442,0.060771,0.241706,0.238038,0.788271,0.996142,0.372254,0.423402,0.108967,0.056507,0.610980,0.470933,0.433565,0.844938,0.410690,0.796849,0.829519,0.961372,0.883254,0.856374,0.890140
129,0.962811,0.673151,0.065105,0.080099,0.249843,0.792128,0.997106,0.369458,0.431820,0.082261,0.056634,0.541967,0.624274,0.603493,0.837060,0.432879,0.795888,0.836671,0.956544,0.889819,0.863052,0.900472
130,0.946237,0.699949,0.069099,0.033180,0.266770,0.795984,0.998071,0.371547,0.470637,0.050960,0.057423,0.490124,0.716485,0.721753,0.832006,0.455069,0.794926,0.843822,0.951716,0.896384,0.869730,0.912345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.692912,0.000000,0.030808,0.075260,0.381401,0.987600,0.828698,0.005731,0.363180,0.185517,0.034447,0.473616,0.382418,0.412205,0.721547,0.232479,0.768057,0.988798,0.647567,0.992324,0.990966,0.843701
188,0.693045,0.029504,0.027720,0.052572,0.466609,0.990700,0.823090,0.037761,0.457237,0.124386,0.045597,0.583810,0.522215,0.496420,0.710959,0.228761,0.756242,0.987198,0.649169,0.991228,0.989675,0.850718
189,0.701363,0.043125,0.023074,0.013826,0.530936,0.993800,0.817482,0.087044,0.484434,0.111254,0.053649,0.580652,0.685920,0.639336,0.691975,0.225042,0.744427,0.985598,0.650771,0.990131,0.988384,0.864656
190,0.702586,0.059756,0.017547,0.056080,0.611274,0.996900,0.811875,0.118264,0.530068,0.099285,0.056431,0.585672,0.758157,0.707155,0.691303,0.221324,0.732612,0.983998,0.652373,0.989035,0.987094,0.878935


In [38]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 22)

In [39]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [40]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [41]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [42]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1916744497, 377446890, 403871850, 1741787061, 3765093672, 2300779394, 2748423680, 1966600773, 1923073488, 3345492130, 528392471, 3291986566, 2464152914, 1622183764, 1787305456, 1940790172, 204831517, 3289186794, 2040117415, 1685455330]


Step: 0 ___________________________________________
val_loss: 111.9434814453125
winner_seed: 1916744497


Step: 1 ___________________________________________
val_loss: 95.50990295410156
winner_seed: 377446890


Step: 2 ___________________________________________
val_loss: 111.70528411865234


Step: 3 ___________________________________________
val_loss: 111.24243927001953


Step: 4 ___________________________________________
val_loss: 114.40489959716797


Step: 5 ___________________________________________
val_loss: 115.279296875


Step: 6 ___________________________________________
val_loss: 114.01253509521484


Step: 7 ___________________________________________
val_loss: 109.90316009521484


Step: 8 ___________________________________________


2023-10-20 15:14:43.379394: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 24.528419494628906
winner_seed: 1923073488


Step: 9 ___________________________________________


2023-10-20 15:16:45.856731: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 89.71253204345703


Step: 10 ___________________________________________
val_loss: 115.4365234375


Step: 11 ___________________________________________
val_loss: 73.64981079101562


Step: 12 ___________________________________________


2023-10-20 15:19:41.760729: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 71.89945983886719


Step: 13 ___________________________________________


2023-10-20 15:20:41.423412: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 101.81600952148438


Step: 14 ___________________________________________
val_loss: 83.19852447509766


Step: 15 ___________________________________________
val_loss: 75.08496856689453


Step: 16 ___________________________________________
val_loss: 110.94689178466797


Step: 17 ___________________________________________
val_loss: 59.15398406982422


Step: 18 ___________________________________________
val_loss: 107.73198699951172


Step: 19 ___________________________________________
val_loss: 118.44501495361328


final_seed: 1923073488


In [43]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 129ms/step - loss: 6483.9214 - val_loss: 186.2917
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 523.6993 - val_loss: 183.1983
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 628.3912 - val_loss: 121.2473
Epoch 4/10000
4/4 [==============================] - 0s 11ms/step - loss: 543.2755 - val_loss: 206.6830
Epoch 5/10000
4/4 [==============================] - 0s 11ms/step - loss: 527.4366 - val_loss: 197.2667
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 562.6271 - val_loss: 122.3343
Epoch 7/10000
4/4 [==============================] - 0s 11ms/step - loss: 875.5009 - val_loss: 117.8471
Epoch 8/10000
4/4 [==============================] - 0s 12ms/step - loss: 618.1402 - val_loss: 150.6083
Epoch 9/10000
4/4 [==============================] - 0s 12ms/step - loss: 543.3576 - val_loss: 278.5657
Epoch 10/10000
4/4 [==============================] - 0s 11ms/

4/4 [==============================] - 0s 10ms/step - loss: 640.4011 - val_loss: 207.3192
Epoch 80/10000
4/4 [==============================] - 0s 11ms/step - loss: 448.7526 - val_loss: 602.1103
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 409.0072 - val_loss: 604.1743
Epoch 82/10000
4/4 [==============================] - 0s 10ms/step - loss: 509.4821 - val_loss: 942.0001
Epoch 83/10000
4/4 [==============================] - 0s 12ms/step - loss: 509.0409 - val_loss: 341.5625
Epoch 84/10000
4/4 [==============================] - 0s 12ms/step - loss: 532.2867 - val_loss: 397.1788
Epoch 85/10000
4/4 [==============================] - 0s 11ms/step - loss: 385.4303 - val_loss: 976.4436
Epoch 86/10000
4/4 [==============================] - 0s 12ms/step - loss: 491.2976 - val_loss: 364.6727
Epoch 87/10000
4/4 [==============================] - 0s 10ms/step - loss: 399.8958 - val_loss: 835.0878
Epoch 88/10000
4/4 [==============================] - 0s 11ms/step - l

4/4 [==============================] - 0s 10ms/step - loss: 374.4782 - val_loss: 151.1055
Epoch 157/10000
4/4 [==============================] - 0s 10ms/step - loss: 271.0602 - val_loss: 450.6715
Epoch 158/10000
4/4 [==============================] - 0s 10ms/step - loss: 239.9336 - val_loss: 122.7685
Epoch 159/10000
4/4 [==============================] - 0s 10ms/step - loss: 266.3933 - val_loss: 135.4844
Epoch 160/10000
4/4 [==============================] - 0s 11ms/step - loss: 278.6469 - val_loss: 576.0548
Epoch 161/10000
4/4 [==============================] - 0s 11ms/step - loss: 217.6436 - val_loss: 531.9092
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 298.7747 - val_loss: 281.4564
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 243.4968 - val_loss: 1219.6362
Epoch 164/10000
4/4 [==============================] - 0s 10ms/step - loss: 448.5080 - val_loss: 181.9097
Epoch 165/10000
4/4 [==============================] - 0s 10m

4/4 [==============================] - 0s 10ms/step - loss: 140.8564 - val_loss: 108.9488
Epoch 234/10000
4/4 [==============================] - 0s 11ms/step - loss: 118.3062 - val_loss: 108.1079
Epoch 235/10000
4/4 [==============================] - 0s 11ms/step - loss: 130.2586 - val_loss: 109.4826
Epoch 236/10000
4/4 [==============================] - 0s 11ms/step - loss: 113.2975 - val_loss: 120.6657
Epoch 237/10000
4/4 [==============================] - 0s 10ms/step - loss: 300.3052 - val_loss: 228.3938
Epoch 238/10000
4/4 [==============================] - 0s 10ms/step - loss: 178.2892 - val_loss: 135.9115
Epoch 239/10000
4/4 [==============================] - 0s 11ms/step - loss: 140.6560 - val_loss: 131.0556
Epoch 240/10000
4/4 [==============================] - 0s 10ms/step - loss: 148.8578 - val_loss: 103.9057
Epoch 241/10000
4/4 [==============================] - 0s 10ms/step - loss: 148.9768 - val_loss: 114.5429
Epoch 242/10000
4/4 [==============================] - 0s 12ms

4/4 [==============================] - 0s 11ms/step - loss: 110.0950 - val_loss: 138.7049
Epoch 311/10000
4/4 [==============================] - 0s 11ms/step - loss: 112.7795 - val_loss: 495.9624
Epoch 312/10000
4/4 [==============================] - 0s 11ms/step - loss: 305.2510 - val_loss: 279.9376
Epoch 313/10000
4/4 [==============================] - 0s 11ms/step - loss: 204.1173 - val_loss: 130.6867
Epoch 314/10000
4/4 [==============================] - 0s 12ms/step - loss: 136.2607 - val_loss: 117.4252
Epoch 315/10000
4/4 [==============================] - 0s 11ms/step - loss: 102.7248 - val_loss: 109.7073
Epoch 316/10000
4/4 [==============================] - 0s 11ms/step - loss: 117.0232 - val_loss: 113.8992
Epoch 317/10000
4/4 [==============================] - 0s 10ms/step - loss: 121.1712 - val_loss: 107.8319
Epoch 318/10000
4/4 [==============================] - 0s 10ms/step - loss: 123.0791 - val_loss: 106.2637
Epoch 319/10000
4/4 [==============================] - 0s 10ms

...

4/4 [==============================] - 0s 13ms/step - loss: 27.6700 - val_loss: 64.3661
Epoch 1950/10000
4/4 [==============================] - 0s 13ms/step - loss: 17.2230 - val_loss: 60.2289
Epoch 1951/10000
4/4 [==============================] - 0s 14ms/step - loss: 20.4897 - val_loss: 51.7723
Epoch 1952/10000
4/4 [==============================] - 0s 14ms/step - loss: 17.3500 - val_loss: 64.0832
Epoch 1953/10000
4/4 [==============================] - 0s 14ms/step - loss: 25.4488 - val_loss: 120.5876
Epoch 1954/10000
4/4 [==============================] - 0s 14ms/step - loss: 17.5344 - val_loss: 57.2510
Epoch 1955/10000
4/4 [==============================] - 0s 15ms/step - loss: 17.9807 - val_loss: 50.1157
Epoch 1956/10000
4/4 [==============================] - 0s 15ms/step - loss: 21.2951 - val_loss: 75.8998
Epoch 1957/10000
4/4 [==============================] - 0s 16ms/step - loss: 16.2969 - val_loss: 64.1404
Epoch 1958/10000
4/4 [==============================] - 0s 14ms/step - 

4/4 [==============================] - 0s 16ms/step - loss: 13.2618 - val_loss: 75.6905
Epoch 2028/10000
4/4 [==============================] - 0s 14ms/step - loss: 11.7119 - val_loss: 65.9746
Epoch 2029/10000
4/4 [==============================] - 0s 14ms/step - loss: 15.3510 - val_loss: 73.6030
Epoch 2030/10000
4/4 [==============================] - 0s 14ms/step - loss: 10.9074 - val_loss: 62.5232
Epoch 2031/10000
4/4 [==============================] - 0s 13ms/step - loss: 12.0194 - val_loss: 59.0145
Epoch 2032/10000
4/4 [==============================] - 0s 14ms/step - loss: 11.7754 - val_loss: 74.4012
Epoch 2033/10000
4/4 [==============================] - 0s 15ms/step - loss: 11.1254 - val_loss: 67.9726
Epoch 2034/10000
4/4 [==============================] - 0s 13ms/step - loss: 13.7142 - val_loss: 58.3406
Epoch 2035/10000
4/4 [==============================] - 0s 13ms/step - loss: 13.7030 - val_loss: 62.2500
Epoch 2036/10000
4/4 [==============================] - 0s 12ms/step - l

4/4 [==============================] - 0s 14ms/step - loss: 48.3136 - val_loss: 41.3100
Epoch 2106/10000
4/4 [==============================] - 0s 13ms/step - loss: 48.1534 - val_loss: 44.7405
Epoch 2107/10000
4/4 [==============================] - 0s 14ms/step - loss: 32.4908 - val_loss: 44.4789
Epoch 2108/10000
4/4 [==============================] - 0s 15ms/step - loss: 35.3454 - val_loss: 60.0026
Epoch 2109/10000
4/4 [==============================] - 0s 13ms/step - loss: 31.6360 - val_loss: 56.4141
Epoch 2110/10000
4/4 [==============================] - 0s 14ms/step - loss: 38.6881 - val_loss: 108.7879
Epoch 2111/10000
4/4 [==============================] - 0s 14ms/step - loss: 34.8727 - val_loss: 53.1114
Epoch 2112/10000
4/4 [==============================] - 0s 12ms/step - loss: 28.1527 - val_loss: 54.7143
Epoch 2113/10000
4/4 [==============================] - 0s 13ms/step - loss: 27.6024 - val_loss: 63.0027
Epoch 2114/10000
4/4 [==============================] - 0s 12ms/step - 

4/4 [==============================] - 0s 12ms/step - loss: 15.1327 - val_loss: 75.9850
Epoch 2184/10000
4/4 [==============================] - 0s 14ms/step - loss: 12.2806 - val_loss: 61.2005
Epoch 2185/10000
4/4 [==============================] - 0s 15ms/step - loss: 16.8675 - val_loss: 61.1479
Epoch 2186/10000
4/4 [==============================] - 0s 13ms/step - loss: 13.4765 - val_loss: 85.2665
Epoch 2187/10000
4/4 [==============================] - 0s 13ms/step - loss: 11.7569 - val_loss: 64.8294
Epoch 2188/10000
4/4 [==============================] - 0s 14ms/step - loss: 9.6401 - val_loss: 67.4986
Epoch 2189/10000
4/4 [==============================] - 0s 12ms/step - loss: 14.0982 - val_loss: 67.9016
Epoch 2190/10000
4/4 [==============================] - 0s 13ms/step - loss: 12.6281 - val_loss: 60.5464
Epoch 2191/10000
4/4 [==============================] - 0s 13ms/step - loss: 17.3879 - val_loss: 70.0409
Epoch 2192/10000
4/4 [==============================] - 0s 13ms/step - lo

4/4 [==============================] - 0s 13ms/step - loss: 15.0467 - val_loss: 81.0122
Epoch 2262/10000
4/4 [==============================] - 0s 12ms/step - loss: 13.9300 - val_loss: 79.1122
Epoch 2263/10000
4/4 [==============================] - 0s 14ms/step - loss: 11.5652 - val_loss: 79.9810
Epoch 2264/10000
4/4 [==============================] - 0s 12ms/step - loss: 13.1207 - val_loss: 77.8682
Epoch 2265/10000
4/4 [==============================] - 0s 14ms/step - loss: 18.8004 - val_loss: 66.5184
Epoch 2266/10000
4/4 [==============================] - 0s 14ms/step - loss: 12.3409 - val_loss: 70.6783
Epoch 2267/10000
4/4 [==============================] - 0s 14ms/step - loss: 13.8902 - val_loss: 69.3996
Epoch 2268/10000
4/4 [==============================] - 0s 14ms/step - loss: 11.2412 - val_loss: 65.4940
Epoch 2269/10000
4/4 [==============================] - 0s 13ms/step - loss: 12.3931 - val_loss: 60.0839
Epoch 2270/10000
4/4 [==============================] - 0s 14ms/step - l

In [44]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [45]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 17ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,113.645241,112.510719,109.402443,115.312042,119.309052,99.311165,89.827652,92.694572,97.354691,103.488747,111.233276,114.654991,111.50338,116.349648,130.642197,125.347267,115.79789,108.847282,109.018272,97.818001,95.688194,102.234077,109.733124,118.642769,122.225563,124.99749,125.548111,122.240677,126.69355,113.884422
Target,120.055,125.769,112.904,123.201,112.725,90.828,96.36,86.444,94.285,98.986,88.072,141.01,122.652,142.145,121.124,130.503,104.115,90.69,102.685,96.144,102.197,106.712,124.057,124.625,133.116,144.31,140.357,152.769,124.038,95.054
Error,6.40976,13.258278,3.501556,7.888954,6.584053,8.483162,6.532349,6.250572,3.069687,4.502747,23.161278,26.355003,11.148621,25.795357,9.518196,5.155739,11.682892,18.15728,6.333275,1.674004,6.508804,4.477921,14.323875,5.982231,10.890434,19.312508,14.808884,30.52832,2.655548,18.830421


In [46]:
display(mae)
display(mape)

11.126056

0.09550884

In [47]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [48]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[1278.7446]] - Target[1290.639]| =  Error: [[11.894409]]; MAPE:[[0.00921591]]
1/1 [==============================] - 0s 17ms/step
Ano-0: |Prediction[[1341.6222]] - Target[1367.6490000000001]| =  Error: [[26.026855]]; MAPE:[[0.01903036]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[735.5898]] - Target[789.644]| =  Error: [[54.0542]]; MAPE:[[0.06845389]]


[array([[11.894409]], dtype=float32),
 array([[26.026855]], dtype=float32),
 array([[54.0542]], dtype=float32)]

30.658487

0.032233384